In [5]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# 加载数据
data = pd.read_csv('stock_data.csv')  # 确保文件路径是正确的
data.sort_index(ascending=False, inplace=True)
data = data.reset_index(drop=True)
data.columns = ['price', 'vol']

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

train_data = data_scaled

def generator(data, lookback=10, delay=1, batch_size=32):
    max_index = len(data) - delay - 1
    i = lookback
    while 1:
        if i + delay >= max_index:
            i = lookback
        samples = np.zeros((batch_size, lookback, data.shape[-1]))
        targets = np.zeros((batch_size, data.shape[-1]))  # 生成器的目标现在包括所有特征
        for j in range(batch_size):
            if i + delay >= max_index:
                i = lookback
            rows = np.arange(i - lookback, i)
            samples[j] = data[rows]
            targets[j] = data[i + delay - 1, :]  # 预测所有特征，而不仅仅是价格
            i += 1
        yield samples, targets

lookback = 50
delay = 1
batch_size = 32
train_gen = generator(train_data, lookback=lookback, delay=delay, batch_size=batch_size)
# test_gen = generator(test_data, lookback=lookback, delay=delay, batch_size=batch_size)

model_path = 'stock_model.h5'  # 确保路径是正确的

if os.path.exists(model_path):
    # 加载模型
    model = load_model(model_path)
else:
    # 定义模型
    model = Sequential()
    model.add(LSTM(32, input_shape=(lookback, data_scaled.shape[-1])))
    model.add(Dense(data_scaled.shape[-1]))  # 输出层的单元数量现在匹配特征的数量
    model.compile(optimizer=Adam(), loss='mse')

    # model.fit(train_gen, epochs=20, steps_per_epoch=500, validation_data=test_gen, validation_steps=50)
    model.fit(train_gen, epochs=60, steps_per_epoch=30)
    # 保存模型
    model.save(model_path)

# 对未来500个价格数据进行预测
future = 100
data_to_predict = data_scaled[-lookback:]  # 最后lookback个数据点
predicted_data = []
for _ in range(future):
    samples = np.reshape(data_to_predict[-lookback:], (1, lookback, data_scaled.shape[-1]))
    predictions = model.predict(samples)
    last_prediction = predictions[0, :]  # 获取所有预测特征
    predicted_data.append(last_prediction)
    data_to_predict = np.vstack([data_to_predict[1:], [last_prediction]])  # 使用所有预测特征

# 对最后500个预测结果进行反归一化
predicted_data = np.array(predicted_data)
predicted_data = scaler.inverse_transform(predicted_data)
predicted_data[:, 1] = np.maximum(predicted_data[:, 1], 0)


# 保存预测结果到csv文件
predicted_df = pd.DataFrame(predicted_data, columns=['predicted_price', 'predicted_vol'])
predicted_df.to_csv('predicted_data_lstm.csv', index=False)


Epoch 1/60


2023-06-29 15:08:36.876448: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


30/30 [==============================] - 1s 7ms/step - loss: 0.3493
Epoch 2/60
30/30 [==============================] - 0s 7ms/step - loss: 0.6213
Epoch 3/60
30/30 [==============================] - 0s 6ms/step - loss: 0.2011
Epoch 4/60
30/30 [==============================] - 0s 6ms/step - loss: 1.3613
Epoch 5/60
30/30 [==============================] - 0s 6ms/step - loss: 0.5176
Epoch 6/60
30/30 [==============================] - 0s 6ms/step - loss: 0.1616
Epoch 7/60
30/30 [==============================] - 0s 6ms/step - loss: 0.1114
Epoch 8/60
30/30 [==============================] - 0s 6ms/step - loss: 0.1360
Epoch 9/60
30/30 [==============================] - 0s 6ms/step - loss: 0.2452
Epoch 10/60
30/30 [==============================] - 0s 6ms/step - loss: 0.1139
Epoch 11/60
30/30 [==============================] - 0s 6ms/step - loss: 0.6162
Epoch 12/60
30/30 [==============================] - 0s 6ms/step - loss: 0.3896
Epoch 13/60
30/30 [==============================] - 0s 6ms/

/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 10ms/step
